# TAD Week 4: Drug Regression

The goal of this exercise is to introduce you to fitting regression
models, to regression diagnostics and to estimating prediction
error using cross-validation. In addition, we will re-inforce
bootstrapping methods and show how versatile this approach is for
calculating standard errors and confidence intervals.

The scenario: You are a researcher developing new drugs to treat
Amyotrophic Lateral Sclerosis (ALS, a.k.a. "Lou Gehrig's Disease"). Prior
to testing a new ALS drug in SOD1 mice, you need to develop and test an
implantable device to allow chronic delivery of the drug over several
days. You contract with a company to produce custom osmotic mini-pumps
loaded with drug, and they send you samples of devices from three
different manufacturing lots (labeled 'A','B' and 'C'). You implant them
in mice for different lengths of time and then remove them and measure
the amount of drug remaining in the device. You would like to answer the
following questions:

1. How is the drug released over time?
2. How well might our model predict future data?

Other issues to address:
Different lots are likely to differ in uninteresting ways, such as the
initial amount of drug loaded. How can we prevent this from contaminating
our analysis?

The data:
Each row is data from one animal (n = 27)
Column 1 is the manufacturing lot: A, B or C
Column 2 is the length of time the device was implanted, in hours
Column 3 is the amount of drug *remaining* in the device, in mg.

Original source of exercise: Efron, B. & Tibshirani Robert, J. (1993) 
An introduction to the bootstrap. Chapman & Hall, London u.a. 
Ex. 9.3 from E & T, Chapter 9, pp. 107 - 112
Also now includes cross-validation example from Ch. 17

Adapted by RTB, home with the Bubbaloo, 21 Dec. 2016
Developed for homework by RAS and RTB, August 2017. Adapted to Python by EB September 2021.

What to do: Login to learning catalytics and join the session for the
module entitled "Drug Regression". You will answer a series of
questions based on the guided programming below.  Read through and follow the instructions provided.
In some cases you will be asked to answer a question, clearly indicated
by 'QUESTION'. In other cases, you be asked to supply missing code,
indicated by 'TODO'.

**Concepts covered:**
1. plotting grouped data 
2. simple linear regression using sklearn 
3. simple linear regression using statsmodel
4. linear mixed effects models using statsmodel
5. two methods for bootstrapping SE estimates: residuals vs. pairs
6. regression diagnostics: residuals vs. fitted; q-q plot
7. estimating prediction error using cross-validation



In [ ]:
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

matplotlib.rcParams.update({'font.size': 16})

# Load and plot data

In [ ]:
# Load data
!gdown --id 1YfCMa4oWfXTWWRQ3C43I-Xbf-zqY_VAT

ds = pd.read_excel('/content/DrugData.xlsx')

ds.head()

In [ ]:
# Plot data with different symbols for the different lots

fig, ax = plt.subplots(1, 1, figsize = (7, 7))
sns.scatterplot(data = ds, x = 'hrs', y = 'amount', hue = 'lot', style = 'lot', ax = ax)

ax.set(xlabel = 'Time implanted (hrs)', 
       ylabel = 'Drug remaining (mg)',
       title = 'Tests of osmotic mini-pump for drug delivery');

**QUESTION (Q1)**: By eye, does the relationship between time implanted and drug remaining look to be linear?


# Simple linear regression

Is there a relationship between the amount of time the drug delivery
device was implanted and the amount of drug remaining in the device? (p.
108). We will perform a simple linear regression using `sklearn`


In [ ]:
from sklearn import linear_model

# Create model
reg_model = linear_model.LinearRegression()

# Fit model
reg_model.fit(X = ds.hrs.values[:, None], y = ds.amount.values)

print(reg_model.coef_)
print(reg_model.intercept_)

# Simple linear regression using statsmodel

`sklearn` is a very popular package for machine learning algorithms, including regression. It's a little light on statistics though, it doesn't compute confidence intervals, t-statistics, standard errors, etc. We can use a library called `statsmodel` instead, which is what we'll use for the rest of this notebook. This has more useful statistics tools related to regression.

We will use `OLS` which stands for ordinary least squares. See the documentation [here](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html)

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

# TODO: Use the function 'OLS' to perform the same regression. Hints: add_constant 
# will add a constant to the input data. 
lin_reg = OLS(...).fit()

You can see a bunch of information about the fit by calling `summary`.

In [ ]:
lin_reg.summary()


**QUESTION (Q2)**: Do the confidence intervals for our beta coefficients indicate a significant linear relationship between amount of time implanted and the amount of drug remaining in the device? (Hint: this info is contained in the summary above but you can also look at `lin_reg.conf_int()`)


In [ ]:
# TODO: extract the value of the y-intercept to a variable called 'b0' and
# the slope to 'b1':
b0 = ...
b1 = ...

# Visualize
fig, ax = plt.subplots(1, 1, figsize = (7, 7))
sns.scatterplot(data = ds, x = 'hrs', y = 'amount', hue = 'lot', style = 'lot', ax = ax)

# Plot the regression line
xlim = ax.get_xlim()
xvals = np.arange(xlim[0], xlim[1], .001)
ax.plot(xvals, lin_reg.predict(add_constant(xvals)), 'k')

ax.set(xlabel = 'Time implanted (hrs)', 
       ylabel = 'Drug remaining (mg)',
       title = 'Tests of osmotic mini-pump for drug delivery');

**QUESTION (Q3)**: What is the regression model estimate of the y-intercept? 
(Write down the model!!!)

**QUESTION (Q4)**: What is its corresponding standard error (can get with `lin_reg.bse`)?

**QUESTION (Q5)**: What is H0?

**QUESTION (Q6)**: Is our y-intercept value statistically significant at
alpha = 0.05?

**QUESTION (Q7)**: Do we care about the value of the y-intercept?


# Linear mixed effects: allowing for different y-intercepts

NOTE: This section is a freebie: you don't actually have to do anything except run the code. The idea is for you to see how it's done but not to spend too much time figuring out the arcana. But there is a question at the end of the section!


Look closely at the first figure in this notebook. Most of the points for Lot C are above the
line; most for A and B are below. This suggests that different lots start
out with slightly different amounts of drug. We don't really care about
this lot-to-lot variation, but it could affect our ability to get a good
estimate of the thing we do care about, which is the slope. To a
statistician, variables that vary "just because" and whose individual
labels (e.g. 'Lot A', 'Lot B', 'Lot C') don't have experimental
significance are referred to as "random effects," whereas variables
that "matter" (experimentally speaking) are referred to as "fixed
effects." Think of it this way, if the labels of the different lots got
switched, it wouldn't much matter to us, but if the 'labels' for the
different time points got switched, it would be a disaster. A linear
model that allows for both 'fixed' and 'random' effects is called a
"linear mixed effects model." We have a fixed effect for the slope (i.e.
'hrs') and the intercept, but we also allow a random addition to the
intercept for each lot.


In [ ]:
from statsmodels.regression.mixed_linear_model import MixedLM

# Use MixedLM to make a linear mixed effects model of the amount of
# drug remaining (dependent variable) with a fixed effect of hours
# remaining and random effect of lot 
lme = MixedLM(ds.amount, add_constant(ds.hrs), groups = ds['lot']).fit(reml = False)

Now we need to read out the individual intercepts from the model

In [ ]:
# Give us the fixed effects (slope & intercept)
fixed_effects = lme.fe_params

# Give us the random effects
random_effects = lme.random_effects

In [ ]:
# Plot the individual regression lines

fig, ax = plt.subplots(1, 1, figsize = (7, 7))
sns.scatterplot(data = ds, x = 'hrs', y = 'amount', hue = 'lot', style = 'lot', ax = ax)

# Plot the regression line
xlim = ax.get_xlim()
xvals = np.arange(xlim[0], xlim[1], .001)
ax.plot(xvals, lin_reg.predict(add_constant(xvals)), 'k', label = 'Lin reg')

# Plot specific regression lines
y_predictions_A = random_effects['A'][0] + fixed_effects['const'] + fixed_effects['hrs']*xvals
ax.plot(xvals, y_predictions_A, label = 'A regression')

y_predictions_B = random_effects['B'][0] + fixed_effects['const'] + fixed_effects['hrs']*xvals
ax.plot(xvals, y_predictions_B, label = 'B regression')

y_predictions_C = random_effects['C'][0] + fixed_effects['const'] + fixed_effects['hrs']*xvals
ax.plot(xvals, y_predictions_C, label = 'C regression')

ax.legend()
ax.set(xlabel = 'Time implanted (hrs)', 
       ylabel = 'Drug remaining (mg)',
       title = 'Tests of osmotic mini-pump for drug delivery');

**QUESTION (Q8)**: Only in Matlab.  Do the different lots have different intercepts? To address the issue of lot differences, we can ask whether any of the random effects for the intercept are significantly different from 0. Annoyingly, `statsmodel` doesn't tell us the confidence intervals for the group-based random intercepts so we will skip this question.

# Regression diagnostics, Part 1

What you do AFTER fitting a regression model is every bit as critical as
what you do before and during. These diagnostics are important for giving
us a visual impression of whether our data meet the assumptions of linear
regression:
1) independence (each point in scatter plot is independent of others)
2) linearity: relationship between x & y is linear
3) homoscedasticity of residuals: residuals have the same variance
4) normality of the residuals

We'll look at two measures:
1) Residuals vs. Fitted: The 'fitted' values are the regresion model's
prediction of our y's given the actual x's, and the residuals are the
differences between our actual y's and our model's predictions.

What we want to see: random scatter and no gross departures from linearity and homoscedasticity.


In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(lin_reg.fittedvalues, lin_reg.resid, 'ko')
ax.plot(ax.get_xlim(), [0, 0], 'r')

ax.set(xlabel = 'Linear Predictor',
       ylabel = 'Residuals', 
       title = 'Residuals vs Fitted');

THOUGHT QUESTIONS: What does the plot of residuals vs. fitted look like? Are our assumptions met?

**QUESTION (Q9)**: What do we mean by 'homoscedasticity'?


# Regression diagnostics, Part 2

Normal quantile plot (Q-Q Plot) of residuals. Recall that one of the
assumptions of regression is that our errors are distributed normally.
How can we assess this? While there are a number of statistical tests to
assess 'normality' of data, they are all rather weak (that is, they lack
statistical power to reject the null when it should be rejected and thus
are not very conservative). But a Q-Q plot gives us a good visual.
What we want to see: points fall on main diagonal


In [ ]:
import statsmodels.api as sm
fig = sm.qqplot(lin_reg.resid)


THOUGHT QUESTIONS: What does the Q-Q Plot look like? Are our assumptions
met? See the next section for a better intuition on what Q-Q plots look
like with matching vs. non-matching distributions.

Statistical tests for normality:
Lilliefors Test: lillietest
Jarque-Bera test: jbtest
One-sample Kolmogorov-Smirnov test: kstest
Anderson-Darling test: adtest
Shapiro-Wilk goodness-of-fit test for normality: download from MathWorks

As stated above, the rap against most of these tests is that they are not
very powerful, which, in this case, means that they are not very
conservative as deciders of normality. How would you get a sense of this
by simulation?


# Bonus on intuition for Q-Q Plot

A quanitle-quantile plot (also called a q-q plot) visually assesses
whether sample data comes from a specified distribution. Alternatively, a
q-q plot assesses whether two sets of sample data come from the same
distribution.

A q-q plot orders the sample data values from smallest to largest, then
plots these values against the expected value for the specified
distribution at each quantile in the sample data. The quantile values of
the input sample appear along the y-axis, and the theoretical values of
the specified distribution at the same quantiles appear along the x-axis.
If the resulting plot is linear, then the sample data likely comes from
the specified distribution.

The q-q plot selects quantiles based on the number of values in the
sample data. If the sample data contains n values, then the plot uses n +
1 quantiles. Plot the ith ordered value (also called the ith order
statistic) against the i (n+1) th quantile of the specified distribution.

A q-q plot can also assesses whether two sets of sample data have the
same distribution, even if you do not know the underlying distribution.
The quantile values for the first data set appear on the x-axis and the
corresponding quantile values for the second data set appear on the
y-axis. Since q-q plots rely on quantiles, the number of data points in
the two samples does not need to be equal. If the sample sizes are
unequal, the q-q plot chooses the quantiles based on the smaller data
set. If the resulting plot is linear, then the two sets of sample data
likely come from the same distribution.




In [ ]:
# Start with a non-normal distribution we have good intuition about:

# TODO: Take 10,000 draws from a uniform discrete random distribution with
# a maximum of 10 and store it in a variable called 'A'
A = ...

fig, axes = plt.subplots(3,1, figsize = (10, 10))
axes[0].hist(A);

# TODO: Make a q-q plot vs. the percentiles in a normal distribution
...

# TODO: Now use qqplot to compare it to the 'right' distribution (uniform)
...

plt.tight_layout()

# Cross-validation to measure prediction error

Calculate the mean residual squared error of our original, simple model (i.e. the one without independent y-intercepts for the different lots).
This is just the summed squared error divided by the number of data points.


In [ ]:
mean_RSE = sum(lin_reg.resid**2) / ds.shape[0]
print(mean_RSE)

The `mean_RSE` is a measure of how well our model describes the data. But it
is overly optimistic, because it is measuring performance using the SAME
data that were used to fit the model. That is, the model is optimized to
fit precisely this data. But how well would it do on another data set?
Well, we could either repeat the experiment, and see how well the model
fit to the first data set predicted the new experimental values.
Alternatively, we can use cross-validation, in which we divide up the
data into K equal-sized parts, fit the model to the other K-1 parts, then
measure the error in predicting the Kth part. This is called K-fold
cross-validation. A common method is K=n, referred to as "leave-one-out"
cross-validation.

Note that we can no longer use `lin_reg.fittedvalues` to get predicted values, since we want to assess residuals on new data. You can use `lin_reg.predict` to get predictions to inputted data.


In [ ]:
# TODO: Perform a leave-one-out cross-validation in order to calculate 
#  our CV_residuals

n_pts = ds.shape[0] # number of data points
CV_residuals = np.zeros((n_pts,))

for k in range(n_pts):
    ... # your code here!
    
    # Compute RSE
    CV_residuals[k] = ...

In [ ]:
# TODO: Calculate the mean squared error of our cross-validated residuals.
CV_mse = ...
print(CV_mse)

**QUESTION (Q10)**: What is the cross-validated mean squared error?

**QUESTION (Q11)**: By how many percent does `mean_RSE` underestimate the prediction error as computed by cross-validation? Use `CV_mse` as the gold standard.

In [ ]:
underest_percent = ...
print(underest_percent)

# Compare actual residuals with the residuals obtained by cross-validation


In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(ds.hrs, lin_reg.resid, 'ko', label = 'Full-model residuals')
ax.plot(ds.hrs, CV_residuals, 'r*', label = 'Cross-validated residuals')
ax.set(xlabel = 'Time implanted (hrs)', 
       ylabel = 'residual',
       title = 'Full-model vs. CV Residuals');

**THOUGHT QUESTION**: How similar are the actual and cross-validated
residuals? In the plot, each asterisk has a corresponding circle. Try to
put into words what the difference is between the asterisk and its
corresponding circle.

# Other estimates of prediction error

As covered on  pp. 242-243 of E&T, other measures of prediction error
include adjusted RSE, Akaike Information Criterion (AIC) and Bayesian
Information Criterion (BIC). 




In [ ]:
# TODO: Read up on 'Akaike Information Criterion', then figure out how to 
# derive these values for our two models. HINT: We get these for free from
# both 'OLS' and `MixedLM`  


**QUESTION (Q12)**: Based on the AIC values, which is the "better" model?


# Application of the bootstrap to regression models

You might be asking, why do we need to bootstrap standard errors for
regression models when we get these (and more) for free from the GLM. The
answer is that we don't. BUT, there are plenty of other situations in
which we might need to leave the safe, cozy domain of the GLM to
situations in which our regression function is non-linear in the
parameters (the beta's) or when we want to use a fitting method other
than least squares. There is a good example of this in the extra exercise
'etCellSurvivalReg.m' where we use 'least median squares' for a more
robust fit in the presence of fishy data (i.e. outliers).

Classic quote from E&T: "Thus reassured that the bootstrap is giving
reasonable answers in a case we can analyze mathematically, we can go on
to apply the bootstrap to more general regression models that have no
mathematical solution: where the regression function is non-linear in the
parameters beta, and where we use fitting methods other than
least-squares."

So, in the next two sections, we'll use two different bootstrapping
approaches for regression models where we can compare our bootstrap
estimates of standard errors to those we get from the GLM.


## Method 1: Bootstrapping residuals

The basic idea is that we need an estimate of both the regression
coefficients (beta's) and the probability density function (PDF) of the
error terms (F). So we use our estimate of beta to calculate the
approximate errors, e_i = y_i - BX. These are just the residuals. In
other words, our estimate of F is the empirical distribution of the
residuals!

With OLS in statsmodel, we get estimates of the standard error for each of our coefficients.


In [ ]:
lin_reg.bse

We also get everything we need for the bootstrap:
The fitted values (i.e. the predicted values for our actual x-values) (plotted below) and the residuals in `lin_reg.resid`

In [ ]:
# Plot the individual regression lines

fig, ax = plt.subplots(1, 1, figsize = (7, 7))
sns.scatterplot(data = ds, x = 'hrs', y = 'amount', hue = 'lot', style = 'lot', ax = ax)

# Plot the regression line
xlim = ax.get_xlim()
xvals = np.arange(xlim[0], xlim[1], .001)
ax.plot(xvals, lin_reg.predict(add_constant(xvals)), 'k')

# Plot specific regression lines
y_predictions_A = random_effects['A'][0] + fixed_effects['const'] + fixed_effects['hrs']*xvals
plt.plot(xvals, y_predictions_A)

y_predictions_B = random_effects['B'][0] + fixed_effects['const'] + fixed_effects['hrs']*xvals
plt.plot(xvals, y_predictions_B)

y_predictions_C = random_effects['C'][0] + fixed_effects['const'] + fixed_effects['hrs']*xvals
plt.plot(xvals, y_predictions_C)

# Plot the predicted values for x-values
plt.plot(ds.hrs, lin_reg.fittedvalues, 'ko')

ax.set(xlabel = 'Time implanted (hrs)', 
       ylabel = 'Drug remaining (mg)',
       title = 'Tests of osmotic mini-pump for drug delivery');

TO DO: Write the 'for' loop that will bootstrap the residuals to allow us
to estimate our regression coefficients (betas). We've already run the
regression on our data above, giving us a model predictions (fitted data)
and residuals. In each iteration of the loop, we will randomly resample
from the total pool of possible residuals (use `lin_reg.resid`),
giving us errorStar (see below). For each of the `n_boot` iterations, we
will draw the `n_pts` residuals, which is the number of data points in our
original dataset, with replacement. We will add these residuals to the
fitted linear predictors of the model to get `y_star`, which we can think of
like a 'recomputation' of our data. Another way to think about this is
`y_star[i] = beta_hat_0x[i] + error_star[i]` Then, recompute the regression,
only with `y_star` instead of the original Y. Store the regression
coefficients from each run in a row of `all_beta` (`all_beta` should be a
n_boot-x-2 matrix if completed correctly). 


In [ ]:
n_boot = 1000;
all_beta = np.zeros((n_boot, 2))

np.random.seed(123)
for k in range(n_boot):
    y_star = ...
    
    model_fit = OLS(y_star, add_constant(ds.hrs.values)).fit()
    all_beta[k, :] = model_fit.params

Then we can take the standard deviation of our "new" coefficients to estimate standard error.

In [ ]:
bs_SE_resid = ...

print(bs_SE_resid)

**THOUGHT QUESTION** (no LC component): Compare `bs_SE_resid` with
`lin_reg.bse`. How similar are they?

**QUESTION (Q13)**: What is your estimate of the standard error of the slope
coefficient based on bootstrapping residuals?


## Method 2: Bootstrapping pairs

Instead of resampling residuals and applying them to fitted data, we can
select random pairs (or cases) of the data, keeping the x's and y's
matched. That is, if we picture the row (observations) by columns
(variables) structure of our data, we are randomly sampling rows.
We then perform regression on these bootstrapped pairs. 

In [ ]:
np.random.seed(123)
for k in range(n_boot):
    ... # your code here!
    all_beta[k, :] = ...

# TODO: Calculate the standard errors for this method:
bs_SE_pairs = ...
print(bs_SE_pairs)

**QUESTION (Q14)**: What is your estimate of the standard error of the slope coefficient based on bootstrapping pairs?


## Which method is better?

E & T state that "Bootstrapping pairs is less sensitive to assumptions
than bootstrapping residuals." BS of residuals assumes that the error
distribution (i.e. residuals) does not depend on x_i, and this is not
always the case. It depends a lot on how good our assumption of linearity
is and on the homoscedasticity of the data. See fig. 9.2 on p. 114 of E&T